In [1]:
using Lux, Optimisers, Random, Statistics, Zygote
using CairoMakie, MakiePublication
using PyCall, NPZ
using Flux: params

np = pyimport("numpy")

PyObject <module 'numpy' from '/opt/homebrew/opt/python@3.11/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/__init__.py'>

In [4]:
nk = 100
nz = 20
nc = 2000

2000

In [10]:
input = npzread("../files/input_neu_sym_Vh_100.npy")
output = npzread("../files/output_neu_sym_Vh_100.npy")

100×2000 Matrix{Float64}:
 0.0055452   0.00420238  0.00320454  …  2.09809e13  1.91184e13  1.74928e13
 0.00618531  0.00468762  0.00357466     2.96631e13  2.70296e13  2.47311e13
 0.00687286  0.00520882  0.00397221     4.14293e13  3.77509e13  3.45403e13
 0.00760954  0.00576726  0.00439818     5.72083e13  5.21285e13  4.76948e13
 0.00839705  0.00636424  0.00485354     7.81614e13  7.12206e13  6.51626e13
 0.00923708  0.00700104  0.00533928  …  1.05732e14  9.63419e13  8.81467e13
 0.0101313   0.00767895  0.00585638     1.41696e14  1.29112e14  1.18129e14
 0.0110815   0.00839924  0.00640582     1.88234e14  1.71516e14  1.56924e14
 0.0120892   0.00916321  0.00698857     2.47997e14  2.2597e14   2.06745e14
 0.0131563   0.00997214  0.00760562     3.24192e14  2.95396e14  2.70263e14
 0.00545367  0.00413755  0.00315819  …  1.96395e13  1.79061e13  1.63924e13
 0.00608319  0.00461532  0.003523       2.77421e13  2.52933e13  2.31548e13
 0.00675936  0.0051285   0.00391486     3.87138e13  3.52962e13  3.23118e13

In [11]:
x = input'
y = output'

# standardizzazione y
y = (y .- minimum(y, dims=1)) ./ (maximum(y, dims=1) .- minimum(y, dims=1))


2000×100 Matrix{Float64}:
 1.36948e-17  1.0803e-17   8.59351e-18  …  4.88479e-18  4.08245e-18
 1.0223e-17   8.06444e-18  6.41519e-18     3.68234e-18  3.07757e-18
 7.64307e-18  6.02938e-18  4.7964e-18      2.77587e-18  2.32002e-18
 5.77704e-18  4.55742e-18  3.62551e-18     2.11445e-18  1.76724e-18
 4.42753e-18  3.49287e-18  2.77869e-18     1.63244e-18  1.36439e-18
 3.43896e-18  2.71303e-18  2.15834e-18  …  1.27668e-18  1.06706e-18
 2.70169e-18  2.13143e-18  1.69566e-18     1.00935e-18  8.43637e-19
 2.14122e-18  1.68929e-18  1.34394e-18     8.04628e-19  6.72531e-19
 1.70721e-18  1.3469e-18   1.07156e-18     6.44954e-19  5.39076e-19
 1.36532e-18  1.07718e-18  8.56982e-19     5.18299e-19  4.33216e-19
 1.09175e-18  8.61354e-19  6.85286e-19  …  4.16288e-19  3.47953e-19
 8.69763e-19  6.86221e-19  5.45957e-19     3.32988e-19  2.78329e-19
 6.87339e-19  5.42299e-19  4.31457e-19     2.64125e-19  2.20771e-19
 ⋮                                      ⋱               
 0.164454     0.164446     0.1644

In [12]:
print("x: ", size(x), '\t', "y: ", size(y))

x: (2, 100)	y: (2000, 100)

In [13]:
layer_size = 64
in_size = size(x)[1]
out_size = size(y)[1]

nn = Chain(
    Dense(in_size => layer_size, tanh),
    Dense(layer_size => layer_size, tanh),
    Dense(layer_size => layer_size, tanh),
    Dense(layer_size => layer_size, tanh),
    Dense(layer_size => layer_size, tanh),
    Dense(layer_size => out_size)
)


Chain(
    layer_1 = Dense(2 => 64, tanh_fast),  # 192 parameters
    layer_2 = Dense(64 => 64, tanh_fast),  # 4_160 parameters
    layer_3 = Dense(64 => 64, tanh_fast),  # 4_160 parameters
    layer_4 = Dense(64 => 64, tanh_fast),  # 4_160 parameters
    layer_5 = Dense(64 => 64, tanh_fast),  # 4_160 parameters
    layer_6 = Dense(64 => 2000),        # 130_000 parameters
)         # Total: 146_832 parameters,
          #        plus 0 states.

In [14]:
opt = Adam(0.03f0)

Adam(0.03, (0.9, 0.999), 1.0e-8)

In [15]:
rng = MersenneTwister()
Random.seed!(rng, 12345)

MersenneTwister(12345)

In [16]:
tstate = Lux.Training.TrainState(rng, nn, opt)

┌ Warning: No functional GPU backend found! Defaulting to CPU.
│ 
│ 1. If no GPU is available, nothing needs to be done.
│ 2. If GPU is available, load the corresponding trigger package.
│     a. LuxCUDA.jl for NVIDIA CUDA Support.
│     b. LuxAMDGPU.jl for AMD GPU ROCM Support.
│     c. Metal.jl for Apple Metal GPU Support.
└ @ LuxDeviceUtils ~/.julia/packages/LuxDeviceUtils/FNTXQ/src/LuxDeviceUtils.jl:212


Lux.Experimental.TrainState{Chain{@NamedTuple{layer_1::Dense{true, typeof(tanh_fast), typeof(glorot_uniform), typeof(zeros32)}, layer_2::Dense{true, typeof(tanh_fast), typeof(glorot_uniform), typeof(zeros32)}, layer_3::Dense{true, typeof(tanh_fast), typeof(glorot_uniform), typeof(zeros32)}, layer_4::Dense{true, typeof(tanh_fast), typeof(glorot_uniform), typeof(zeros32)}, layer_5::Dense{true, typeof(tanh_fast), typeof(glorot_uniform), typeof(zeros32)}, layer_6::Dense{true, typeof(identity), typeof(glorot_uniform), typeof(zeros32)}}, Nothing}, @NamedTuple{layer_1::@NamedTuple{weight::Matrix{Float32}, bias::Matrix{Float32}}, layer_2::@NamedTuple{weight::Matrix{Float32}, bias::Matrix{Float32}}, layer_3::@NamedTuple{weight::Matrix{Float32}, bias::Matrix{Float32}}, layer_4::@NamedTuple{weight::Matrix{Float32}, bias::Matrix{Float32}}, layer_5::@NamedTuple{weight::Matrix{Float32}, bias::Matrix{Float32}}, layer_6::@NamedTuple{weight::Matrix{Float32}, bias::Matrix{Float32}}}, @NamedTuple{layer_1

In [17]:
vjp_rule = Lux.Training.AutoZygote()

ADTypes.AutoZygote()

In [18]:
function loss_function(model, ps, st, data)
    y_pred, st = Lux.apply(model, data[1], ps, st)
    mse_loss = mean(abs2, y_pred .- data[2])
    return mse_loss, st, ()
end

loss_function (generic function with 1 method)

In [19]:
function main(tstate::Lux.Experimental.TrainState, vjp, data, epochs)
    # data = data .|> gpu_device()
    for epoch in 1:epochs
        grads, loss, stats, tstate = Lux.Training.compute_gradients(
            vjp, loss_function, data, tstate)
        println("Epoch: $(epoch)\t|| Loss: $(loss)")
        tstate = Lux.Training.apply_gradients(tstate, grads)
    end
    return tstate
end

dev_cpu = cpu_device()

tstate = main(tstate, vjp_rule, (x, y), 250)
y_pred = dev_cpu(Lux.apply(tstate.model, x, tstate.parameters, tstate.states)[1])

Epoch: 1	|| Loss: 0.002832305816359235
Epoch: 2	|| Loss: 0.004794318909954356
Epoch: 3	|| Loss: 0.47515854605393987
Epoch: 4	|| Loss: 0.7340083808490466
Epoch: 5	|| Loss: 0.057161317368413386
Epoch: 6	|| Loss: 0.18432009675953223
Epoch: 7	|| Loss: 0.02877649975387729
Epoch: 8	|| Loss: 0.502462871224478
Epoch: 9	|| Loss: 0.33877269833085066
Epoch: 10	|| Loss: 0.1144766315830105
Epoch: 11	|| Loss: 0.07495077804376002
Epoch: 12	|| Loss: 0.035356634988113596
Epoch: 13	|| Loss: 0.11963525321315105
Epoch: 14	|| Loss: 0.048837769113728585
Epoch: 15	|| Loss: 0.025539591640547733
Epoch: 16	|| Loss: 0.05154489513452936
Epoch: 17	|| Loss: 0.053287166932634834
Epoch: 18	|| Loss: 0.018816666264048876
Epoch: 19	|| Loss: 0.029037616294111123
Epoch: 20	|| Loss: 0.038922209609557085
Epoch: 21	|| Loss: 0.020083722651660228
Epoch: 22	|| Loss: 0.021316762714488895
Epoch: 23	|| Loss: 0.02105280220738864
Epoch: 24	|| Loss: 0.022083059669362547
Epoch: 25	|| Loss: 0.013362293495350432
Epoch: 26	|| Loss: 0.014

2000×100 Matrix{Float64}:
 -0.000147648  -6.3337e-5     1.56105e-5   …   6.96519e-5    0.000125582
  0.000247425   0.000156936   6.84495e-5      -0.000177517  -0.000260514
 -0.0002891    -0.000128328   2.71618e-5       0.000156032   0.000291043
 -0.000400779  -0.000223564  -5.20148e-5       0.000251393   0.000400971
 -0.000145626  -0.000123345  -9.93497e-5       0.000113251   0.000142835
 -0.000248291  -0.00011888    6.64057e-6   …   0.000141498   0.000253597
  0.000312208   0.000194239   7.90641e-5      -0.00021064   -0.000314882
  0.00013148    2.71912e-5   -7.21439e-5      -5.93596e-5   -0.000142195
  2.05918e-6   -3.61742e-5   -7.14286e-5       2.25224e-5   -1.58277e-6
  0.000493756   0.000291875   9.25302e-5      -0.000316979  -0.00050764
  0.000118045   0.000153113   0.000184613  …  -0.000116544  -9.33197e-5
 -0.000474337  -0.000305706  -0.000141166      0.000315813   0.000461859
 -0.000180209  -8.25657e-5    1.06679e-5       0.000108375   0.000185729
  ⋮                         

In [20]:
ps, st = Lux.setup(rng, nn)
y_pred, st = Lux.apply(nn, input, ps, st)

print(size(y_pred))

LoadError: DimensionMismatch: A has dimensions (64,2) but B has dimensions (100,2)

In [5]:
input = npzread("../files/train_in_emul_" * string(nc) * ".npy")
output = npzread("../files/train_out_emul_" * string(nc) * ".npy")

pkz = npzread("../files/train_pkz_emul_" * string(nc) * ".npy")
mu2 = npzread("../files/train_mu2_emul_" * string(nc) * ".npy")
prim = npzread("../files/train_prim_emul_" * string(nc) * ".npy")
tf2 = npzread("../files/train_tf2_emul_" * string(nc) * ".npy")

println(size(output))

(2000, 2000)


In [6]:
prova = pkz./(prim .* tf2 .* mu2)
prova = np.reshape(prova, [nc,nk*nz])

prova-output

2000×2000 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

In [9]:
input = npzread("../files/train_in_emul_" * string(nc) * ".npy")
output = npzread("../files/train_pkz_emul_" * string(nc) * ".npy")

x = input
y = np.reshape(output, [nc,nk*nz])'

print(typeof(y))

LinearAlgebra.Adjoint{Float64, Matrix{Float64}}